In [1]:
# Import modules
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from selenium import webdriver
import pandas as pd
import time
from secret import username, password
from sqlalchemy import create_engine
import csv
import numpy as np
import html5lib

### Mac (Irina) Connection

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
# executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
# browser = Browser('chrome', **executable_path, headless=False)

### Windows (Clay) Connection

In [4]:
# executable_path = {'executable_path': 'C:/Users/Clay/chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

### Dataset 1 - Web Scrape Wikipedia Most Costliest hurricanes

In [5]:
url = 'https://en.wikipedia.org/wiki/List_of_costliest_Atlantic_hurricanes'
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)
time.sleep(3)
print("Open web browser")
tables = pd.read_html(url)
most_costly_hurricanes = tables[0]
print("Read and save table")
browser.quit()
most_costly_hurricanes.head()

Open web browser
Read and save table


,Name,Nominal damage(Billions USD),Normalized damage(Billions USD),Season,Storm classificationat peak intensity,Areas affected,References
0,Katrina,$125.0,$116.9,2005,Category 5 hurricane,Louisiana Mississippi The Bahamas United State...,[4][5]
1,Harvey,$125.0,$62.2,2017,Category 4 hurricane,Texas Louisiana South America Central America ...,[5][6]
2,Maria,$90.0,NaN,2017,Category 5 hurricane,Puerto Rico Lesser Antilles Greater Antilles C...,[7]
3,Irma,$77.2,$31.0,2017,Category 5 hurricane,Lesser Antilles Greater Antilles Caribbean Sea...,[5]
4,Sandy,$68.7,$73.5,2012,Category 3 hurricane,The Caribbean United States East Coast Eastern...,[5][8]


In [6]:
# Create a copy of the df
renamed_cost_df = most_costly_hurricanes

In [7]:
# # Rename columns
renamed_cost_df = renamed_cost_df.rename(columns={'Name': 'name','Nominal damage(Billions USD)': 'damage_usd', 'Normalized damage(Billions USD)': 'norm_damage_usd', 
                'Season': 'year', 'Storm classificationat peak intensity': 'category', 'Areas affected':'states'})

In [8]:
# Drop Reference column
renamed_cost_df = renamed_cost_df.drop(columns=['References','states'])

In [9]:
renamed_cost_df.head()

,name,damage_usd,norm_damage_usd,year,category
0,Katrina,$125.0,$116.9,2005,Category 5 hurricane
1,Harvey,$125.0,$62.2,2017,Category 4 hurricane
2,Maria,$90.0,NaN,2017,Category 5 hurricane
3,Irma,$77.2,$31.0,2017,Category 5 hurricane
4,Sandy,$68.7,$73.5,2012,Category 3 hurricane


In [10]:
# Save to the csv file
renamed_cost_df.to_csv("Data/01_most_costly_hurricanes_wiki_web.csv")

In [11]:
# Read csv file into df
renamed_cost_df = pd.read_csv('Data/01_most_costly_hurricanes_wiki_web.csv', index_col=[0])

In [12]:
renamed_cost_df.head()

,name,damage_usd,norm_damage_usd,year,category
0,Katrina,$125.0,$116.9,2005,Category 5 hurricane
1,Harvey,$125.0,$62.2,2017,Category 4 hurricane
2,Maria,$90.0,NaN,2017,Category 5 hurricane
3,Irma,$77.2,$31.0,2017,Category 5 hurricane
4,Sandy,$68.7,$73.5,2012,Category 3 hurricane


## Prepare Cost data

In [13]:
cost_df = renamed_cost_df

In [14]:
cost_df["norm_damage_usd"] = cost_df["norm_damage_usd"].str.replace('$','')
cost_df["norm_damage_usd"] = cost_df["norm_damage_usd"].str.replace('<','')
cost_df["norm_damage_usd"] = cost_df["norm_damage_usd"].str.replace('>','')
cost_df["damage_usd"] = cost_df["damage_usd"].str.replace('$','')
cost_df["damage_usd"] = cost_df["damage_usd"].str.replace('<','')
cost_df["damage_usd"] = cost_df["damage_usd"].str.replace('>','')
cost_df['norm_damage_usd'] = cost_df['norm_damage_usd'].astype(float)
cost_df['norm_damage_usd'] = cost_df['norm_damage_usd'].dropna()
cost_df['damage_usd'] = cost_df['damage_usd'].astype(float)
cost_df['damage_usd'] = cost_df['damage_usd'].dropna()
cost_df = cost_df.dropna()
len(cost_df) # 54 count

54

### Add id column

In [15]:
# Sort table 
cost_df = cost_df.sort_values(by='norm_damage_usd', ascending=False)
cost_df.head()

,name,damage_usd,norm_damage_usd,year,category
0,Katrina,125.0,116.9,2005,Category 5 hurricane
7,Andrew,27.3,106.0,1992,Category 5 hurricane
4,Sandy,68.7,73.5,2012,Category 3 hurricane
1,Harvey,125.0,62.2,2017,Category 4 hurricane
5,Ike,38.0,35.2,2008,Category 4 hurricane


In [16]:
# Reset index
cost_df = cost_df.reset_index(drop=True)
cost_df.head()

,name,damage_usd,norm_damage_usd,year,category
0,Katrina,125.0,116.9,2005,Category 5 hurricane
1,Andrew,27.3,106.0,1992,Category 5 hurricane
2,Sandy,68.7,73.5,2012,Category 3 hurricane
3,Harvey,125.0,62.2,2017,Category 4 hurricane
4,Ike,38.0,35.2,2008,Category 4 hurricane


In [17]:
# Start index from 1
cost_df.index += 1
cost_df.head(2)

,name,damage_usd,norm_damage_usd,year,category
1,Katrina,125.0,116.9,2005,Category 5 hurricane
2,Andrew,27.3,106.0,1992,Category 5 hurricane


In [18]:
cost_df = cost_df.reset_index()
cost_df.head(2)

,index,name,damage_usd,norm_damage_usd,year,category
0,1,Katrina,125.0,116.9,2005,Category 5 hurricane
1,2,Andrew,27.3,106.0,1992,Category 5 hurricane


In [19]:
cost_df = cost_df.rename(columns={'index': 'hurricane_id'})
cost_df.head()

,hurricane_id,name,damage_usd,norm_damage_usd,year,category
0,1,Katrina,125.0,116.9,2005,Category 5 hurricane
1,2,Andrew,27.3,106.0,1992,Category 5 hurricane
2,3,Sandy,68.7,73.5,2012,Category 3 hurricane
3,4,Harvey,125.0,62.2,2017,Category 4 hurricane
4,5,Ike,38.0,35.2,2008,Category 4 hurricane


In [20]:
cost_df.reset_index(drop=True, inplace=True)

In [21]:
cost_df.head()

,hurricane_id,name,damage_usd,norm_damage_usd,year,category
0,1,Katrina,125.0,116.9,2005,Category 5 hurricane
1,2,Andrew,27.3,106.0,1992,Category 5 hurricane
2,3,Sandy,68.7,73.5,2012,Category 3 hurricane
3,4,Harvey,125.0,62.2,2017,Category 4 hurricane
4,5,Ike,38.0,35.2,2008,Category 4 hurricane


In [22]:
# # Rename columns
# total_gross_info = total_gross_info.rename(columns={'Rank': 'rank', 'Title': 'name', 
#                 'Year': 'year', 'GrossNumeric': 'total_revenue_usd'})

In [23]:
# total_gross_info.head()

In [24]:
# Change name column to lower case to easier join 
# total_gross_info['name'] = total_gross_info['name'].str.capitalize()

In [25]:
# # Drop year column
# total_gross_info = total_gross_info.drop(columns='year')

In [26]:
# # Rename Lifetime Gross to total_revenue_usd
# total_gross_info = total_gross_info.rename(columns={'Lifetime Gross': 'total_revenue_usd'})

In [27]:
# total_gross_info.head()

In [28]:
# # Convert total_revenue_usd to Integer
# tot_rev_usd = total_gross_info['total_revenue_usd']
# len(tot_rev_usd)

In [29]:
# list = []
# for i in tot_rev_usd:
#     list.append(i.split('$')[-1]) 
# print(list[:10])

In [30]:
# split_list = []
# for i in list:
#     i = i.split(',')
#     split_list.append(''.join(i)) 

In [31]:
# merged_list = []
# for i in split_list:
#     merged_list.append(''.join(i)) 
# print(merged_list[:5])

In [32]:
# len(merged_list)

In [33]:
# Change revenue column from string to big integer
# total_gross_info['total_revenue_usd'] = merged_list

In [34]:
# total_gross_info['total_revenue_usd'] = pd.to_numeric(total_gross_info['total_revenue_usd'])

In [35]:
# total_gross_info.dtypes

In [36]:
# total_gross_info.to_csv('csv_files/Total_Gross_Info.csv')

## Dataset 2 - Kaggle domestic revenue data - Load and clean¶
#### Please download source data set - https://www.kaggle.com/danielgrijalvas/movies?select=movies.csv
#### Save in 'csv_files/datasets_2745_4700_movies.csv'

In [37]:
# path = 'csv_files/datasets_2745_4700_movies.csv'

In [38]:
# # Convert all rows to UTF-8 type and write to a new file
# with open(path, 'r', encoding='utf-8', errors='ignore') as infile, open(path + '_UTF8.csv', 'w') as outfile:
#      inputs = csv.reader(infile)
#      output = csv.writer(outfile)

#      for index, row in enumerate(inputs):
#          # Create file with no header
#          if index == 0:
#              continue
#          output.writerow(row)

In [39]:
# # Read csv file into df
# domestic_df = pd.read_csv('csv_files/datasets_2745_4700_movies.csv_UTF8.csv')

In [40]:
# # Add header 
# domestic_df.columns = ["budget","company","country","director","genre","gross","name","rating","released","runtime","score","star","votes","writer","year"]

In [41]:
# domestic_df.head(2)

In [42]:
# Make names capitalized
# domestic_df['name'] = domestic_df['name'].str.capitalize()

In [43]:
# domestic_df.head(2)

In [44]:
# # Getting the duplicate movies
# duplicated_domestic_df = domestic_df.loc[domestic_df.duplicated(subset=["name","director"]), "name"].unique()
# duplicated_domestic_df

In [45]:
# # Getting the duplicate movie by title and star released. 
# duplicated_domestic_df = domestic_df.loc[domestic_df.duplicated(subset=["name","star"]), "name"].unique()
# duplicated_domestic_df

In [46]:
# # Getting the duplicate movie by title and country released. 
# duplicated_domestic_df = domestic_df.loc[domestic_df.duplicated(subset=["name","country"]), "name"].unique()
# duplicated_domestic_df

In [47]:
# # Check duplicated movie
# duplicated_domestic_df = domestic_df.loc[domestic_df["name"] == "Godzilla"]
# duplicated_domestic_df

In [48]:
# # Rename gross to domestic_revenue_usd
# domestic_gross_info = domestic_df.rename(columns={'gross': 'domestic_revenue_usd'})

In [49]:
# # Rename released to release_date
# domestic_gross_info = domestic_gross_info.rename(columns={'released': 'release_date'})

In [50]:
# # Drop columns we do not need ('budget', 'runtime', 'score', 'votes', 'year')
# domestic_gross_info = domestic_gross_info.drop(columns=['budget', 'runtime', 'score', 'votes'])

In [51]:
# domestic_gross_info.head(2)

In [52]:
# domestic_gross_info.dtypes

In [53]:
# # Change release_date to datetime type
# domestic_gross_info['release_date'] = pd.to_datetime(domestic_gross_info['release_date'])

In [54]:
# domestic_gross_info.dtypes

In [55]:
# # Write to csv file
# domestic_gross_info.to_csv('csv_files/Domestic_Gross_Info.csv')

In [56]:
# TO THIS POINT, WE HAVE CREATED TWO MASTER DATAFRAMES THAT ARE CLEAN:
# (1) Total_Gross_Info.csv
# (2) Domestic_Gross_Info.csv

### Merge total_gross_info and domestic_gross_info tables on 'name' column

In [57]:
# # combine total and domestic tables on left using name of the movie column
# combined_gross_info = pd.merge(domestic_gross_info, total_gross_info, how='left', on="name")

In [58]:
# combined_gross_info.head()

In [59]:
# len(combined_gross_info)AssertionError

In [60]:
# # Add new column and compute the international_total_revenue
# combined_gross_info['international_revenue_usd'] = \
#         (combined_gross_info['total_revenue_usd'] - combined_gross_info['domestic_revenue_usd'])

In [61]:
# # Rename columns
# combined_gross_info = combined_gross_info.rename(columns={'star':'actor'})

In [62]:
# # Getting the duplicate movie by title and star released. 
# dupl_combined_df = combined_gross_info.loc[combined_gross_info.duplicated(subset=["name","director", "year"]), "name"].unique()
# dupl_combined_df

In [63]:
# # Get all the data for the duplicate movie 
# dupl_df = combined_gross_info.loc[combined_gross_info["name"] == "Clash of the titans"]
# dupl_df

#### (removed) Use pandas to push data into database

In [64]:
# # Check existing tables
# engine.table_names()

In [65]:
# # Create domesic gross revenue table in the database
# domestic_gross_info.to_sql(name='domestic_gross_info', con=engine, if_exists='append', index=False)

In [66]:
# type(total_gross_info['name'])

In [67]:
# total_gross_info

In [68]:
# domestic_gross_info

In [69]:
# # Remove the duplicates
# # clean_combined_df = combined_gross_info.drop_duplicates(subset=['name','director','year'], keep='first')
# clean_combined_df

In [70]:
# len(clean_combined_df)

### Create movies table from combined table

In [71]:
# # Re-arrange the columns and write into new table
# movies = clean_combined_df[['name','total_revenue_usd','international_revenue_usd','domestic_revenue_usd',
#                               'director','actor','writer','genre','rating','company','country','year']]

In [72]:
# movies.head()

In [73]:
# # Fill NaN values with zeroes
# movies[['total_revenue_usd','international_revenue_usd','domestic_revenue_usd']] = \
#     movies[['total_revenue_usd','international_revenue_usd','domestic_revenue_usd']].fillna(0)

### Extract Actor data into a separate table

In [74]:
# # Save star_actor data into a separate table
# actor = movies['actor'].unique()
# len(actor)

In [75]:
# actor[:5]

In [76]:
# actor = pd.DataFrame({'actor': actor})
# actor.head(2)

In [77]:
# actor.index += 1
# actor

In [78]:
# actor = actor.reset_index()
# actor.head(2)

In [79]:
# actor = actor.rename(columns={'index': 'actor_id'})
# actor.head()

In [80]:
# len(actor)

### Merge movie and actor df on actor column

In [81]:
# # combine movie and tables
# movie_actor = pd.merge(movies, actor, how='outer', on="actor")

In [82]:
# # Extract only movie_id and actor_id columns
# movie_actor = movie_actor[['movie_id', 'actor_id']]
# movie_actor

In [83]:
# # Save to csv
# movie_actor.to_csv('csv_files/movie_actor.csv')

### Create director_movie table

In [84]:
# # Save star_actor data into a separate table
# director = movies['director'].unique()
# len(director)

In [85]:
# director[:5]

In [86]:
# director = pd.DataFrame({'director': director})
# director.head(2)

In [87]:
# director.index += 1
# director

In [88]:
# director = director.reset_index()
# director.head(2)

In [89]:
# director = director.rename(columns={'index': 'director_id'})
# director.head()

In [90]:
# len(director)

### Merge movie and director df on actor column

In [91]:
# # combine movie and tables
# movie_director = pd.merge(movies, director, how='outer', on="director")

In [92]:
# # Extract only movie_id and actor_id columns
# movie_director = movie_director[['movie_id', 'director_id']]
# movie_director

In [93]:
# # Save to csv
# movie_director.to_csv('csv_files/movie_director.csv')

### Create company_movie table

In [94]:
# # Save company_movie data into a separate table
# company = movies['company'].unique()
# len(company)

In [95]:
# company[:5]

In [96]:
# company = pd.DataFrame({'company': company})
# company.head(2)

In [97]:
# company.index += 1
# company

In [98]:
# company = company.reset_index()
# company.head(2)

In [99]:
# company = company.rename(columns={'index': 'company_id'})
# company.head()

In [100]:
# len(company)

### Merge movie and company df on actor column

In [101]:
# # combine movie and tables
# company_movie = pd.merge(movies, company, how='outer', on="company")

In [102]:
# # Extract only movie_id and actor_id columns
# company_movie = company_movie[['movie_id', 'company_id']]
# company_movie

In [103]:
# # Save to csv
# company_movie.to_csv('csv_files/company_movie.csv')

## Extract Writer data into a separate table

In [104]:
# # Save writer data into a separate table
# writer = movies['writer'].unique()
# len(writer)

In [105]:
# writer[:5]

In [106]:
# writer = pd.DataFrame({'writer': writer})
# writer.head(2)

In [107]:
# writer.index += 1
# writer

In [108]:
# writer = writer.reset_index()
# writer.head(2)

In [109]:
# writer = writer.rename(columns={'index': 'writer_id'})
# writer.head()

In [110]:
# len(writer)

### Merge writer_id column to movies table

In [111]:
# # combine movie and tables
# movies = pd.merge(movies, writer, how='outer', on="writer")

In [112]:
# movies.head()

## Extract Genre data into a separate table

In [113]:
# # Save genre data into a separate table
# genre = movies['genre'].unique()
# len(genre)

In [114]:
# genre[:5]

In [115]:
# genre = pd.DataFrame({'genre': genre})
# genre.head(2)

In [116]:
# genre.index += 1
# genre

In [117]:
# genre = genre.reset_index()
# genre.head(2)

In [118]:
# genre = genre.rename(columns={'index': 'genre_id'})
# genre.head()

In [119]:
# len(genre)

### Merge genre_id column to movies table

In [120]:
# # combine movies and genre
# movies = pd.merge(movies, genre, how='outer', on="genre")

In [121]:
# movies.head()

In [122]:
# movies.to_csv('csv_files/movies_genre_ids.csv')

## Extract rating data into a separate table

In [123]:
# # Save rating data into a separate table
# rating = movies['rating'].unique()
# len(rating)

In [124]:
# rating[:5]

In [125]:
# rating = pd.DataFrame({'rating': rating})
# rating.head(2)

In [126]:
# rating.index += 1

In [127]:
# rating = rating.reset_index()
# rating.head(2)

In [128]:
# rating = rating.rename(columns={'index': 'rating_id'})
# rating.head()

In [129]:
# len(rating)

### Merge rating_id column to movies table

In [130]:
# # combine movies and rating
# movies = pd.merge(movies, rating, how='outer', on="rating")

In [131]:
# movies.head()

In [132]:
# movies.to_csv('csv_files/movies_rating_ids.csv')

# Remove unnecessary columns from movies

In [133]:
# movies.head(2)

In [134]:
# movies = movies[['movie_id','country','genre_id','writer_id','rating_id','name','year','domestic_revenue_usd',
#                  'international_revenue_usd', 'total_revenue_usd']]

In [135]:
# movies.head()

## Push table to PostgreSQL

### !!! Please import QuickDBD-export.sql in the repo root folder file in pgAdmin
### !!! Run SQL file to generate the tables and reference links
### UNCOMMENT cell below to run the rest of the code

In [136]:
# Connect to the local database
connection_string = f'{username}:{password}@localhost:5432/hurricanes_db'
engine = create_engine(f'postgresql://{connection_string}')

In [137]:
# genre.to_sql(name='genre', con=engine, if_exists='append', index=False)

In [138]:
# writer.to_sql(name='writer', con=engine, if_exists='append', index=False)

In [139]:
# director.to_sql(name='director', con=engine, if_exists='append', index=False)

In [140]:
# rating.to_sql(name='rating', con=engine, if_exists='append', index=False)

In [141]:
# company.to_sql(name='company', con=engine, if_exists='append', index=False)

In [142]:
# actor.to_sql(name='actor', con=engine, if_exists='append', index=False)

In [143]:
# movies.to_sql(name='movies', con=engine, if_exists='append', index=False)

In [144]:
# company_movie.to_sql(name='company_movie', con=engine, if_exists='append', index=False)

In [145]:
# movie_actor.to_sql(name='movie_actor', con=engine, if_exists='append', index=False)

In [147]:
cost_df.to_sql(name="Cost", con=engine, if_exists='append', index=False)